# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234455808459                   -0.50    7.0         
  2   -7.250048123048       -1.81       -1.40    1.0   7.78ms
  3   -7.251224759467       -2.93       -2.12    2.0   8.98ms
  4   -7.251311911525       -4.06       -2.31    2.0   9.78ms
  5   -7.251336080957       -4.62       -2.89    1.0   7.69ms
  6   -7.251338326253       -5.65       -3.25    2.0   9.45ms
  7   -7.251338675410       -6.46       -3.59    1.0   60.0ms
  8   -7.251338748052       -7.14       -3.79    2.0   9.63ms
  9   -7.251338797764       -7.30       -4.53    1.0   8.01ms
 10   -7.251338798605       -9.07       -5.10    2.0   9.47ms
 11   -7.251338798698      -10.03       -5.62    3.0   11.4ms
 12   -7.251338798704      -11.22       -5.99    2.0   10.3ms
 13   -7.251338798704      -12.09       -6.48    1.0   8.57ms
 14   -7.251338798705      -12.95       -6.95    1.0   8.27ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05729580564461938
[ Info: Arnoldi iteration step 2: normres = 0.533186916166295
[ Info: Arnoldi iteration step 3: normres = 0.9300878062177742
[ Info: Arnoldi iteration step 4: normres = 0.26499610240080995
[ Info: Arnoldi iteration step 5: normres = 0.292444096302417
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.96e-02, 3.32e-02, 1.87e-01, 2.20e-01, 2.66e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4726993583276539
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (8.04e-03, 1.59e-01, 4.26e-01, 8.77e-02, 5.14e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09314978248857812
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.18e-04, 3.56e-02, 8.68e-03, 3.44e-02, 7.18e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12101625624088053
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.19e-05